In [1]:
import csv
import pandas as pd
import seaborn as sns
import time
import matplotlib.pyplot as plt

checkout_data = pd.read_csv('data/Checkouts_By_Title_Data_Lens_2017.csv')
checkout_data.info(verbose=True, null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5034542 entries, 0 to 5034541
Data columns (total 6 columns):
BibNumber           5034542 non-null int64
ItemBarcode         5034542 non-null int64
ItemType            5034542 non-null object
Collection          5034542 non-null object
CallNumber          5034464 non-null object
CheckoutDateTime    5034542 non-null object
dtypes: int64(2), object(4)
memory usage: 230.5+ MB


In [2]:
checkout_data.head()

,BibNumber,ItemBarcode,ItemType,Collection,CallNumber,CheckoutDateTime
0,2543647,10063298235,accd,nacd,CD 782.42166 C6606So,01/02/2017 08:13:00 AM
1,3172300,10087522552,acbk,namys,MYSTERY COTTERI 2016,01/02/2017 08:13:00 AM
2,2393405,10054483200,acbk,camys,MYSTERY MAY2006,01/02/2017 08:24:00 AM
3,3199718,10088153514,acdvd,nadvdnf,DVD 781.66092 M3347G 2013,01/02/2017 08:33:00 AM
4,3211526,10089643810,accd,nacd,CD 782.42166 Sh75o,01/02/2017 08:33:00 AM


In [3]:
checkout_data.drop(['ItemBarcode', 'CallNumber'], axis=1, inplace=True)

In [4]:
checkout_data.head()

,BibNumber,ItemType,Collection,CheckoutDateTime
0,2543647,accd,nacd,01/02/2017 08:13:00 AM
1,3172300,acbk,namys,01/02/2017 08:13:00 AM
2,2393405,acbk,camys,01/02/2017 08:24:00 AM
3,3199718,acdvd,nadvdnf,01/02/2017 08:33:00 AM
4,3211526,accd,nacd,01/02/2017 08:33:00 AM


In [5]:
checkout_data[pd.isnull(checkout_data).any(axis=1)]

,BibNumber,ItemType,Collection,CheckoutDateTime


In [6]:
inventory = pd.read_csv('data/Library_Collection_Inventory.csv')
inventory.head()

,BibNum,Title,Author,ISBN,PublicationYear,Publisher,Subjects,ItemType,ItemCollection,FloatingItem,ItemLocation,ReportDate,ItemCount
0,3011076,A tale of two friends / adapted by Ellie O'Rya...,"O'Ryan, Ellie","1481425730, 1481425749, 9781481425735, 9781481...",2014.,"Simon Spotlight,","Musicians Fiction, Bullfighters Fiction, Best ...",jcbk,ncrdr,Floating,qna,09/01/2017,1
1,2248846,"Naruto. Vol. 1, Uzumaki Naruto / story and art...","Kishimoto, Masashi, 1974-",1569319006,"2003, c1999.","Viz,","Ninja Japan Comic books strips etc, Comic book...",acbk,nycomic,NaN,lcy,09/01/2017,1
2,3209270,"Peace, love & Wi-Fi : a ZITS treasury / by Jer...","Scott, Jerry, 1955-","144945867X, 9781449458676",2014.,"Andrews McMeel Publishing,",Duncan Jeremy Fictitious character Comic books...,acbk,nycomic,NaN,bea,09/01/2017,1
3,1907265,The Paris pilgrims : a novel / Clancy Carlile.,"Carlile, Clancy, 1930-",0786706155,c1999.,"Carroll & Graf,","Hemingway Ernest 1899 1961 Fiction, Biographic...",acbk,cafic,NaN,cen,09/01/2017,1
4,1644616,"Erotic by nature : a celebration of life, of l...",NaN,094020813X,"1991, c1988.","Red Alder Books/Down There Press,","Erotic literature American, American literatur...",acbk,canf,NaN,cen,09/01/2017,1


In [7]:
inventory.drop(['ISBN','ItemType', 'ItemCollection', 'FloatingItem', 'ItemLocation'], axis=1, inplace=True)
combined_inv = inventory.groupby(['BibNum', 'Title', 'Author', 'PublicationYear', 'Publisher', 'Subjects','ReportDate'], as_index=False)['ItemCount'].sum()

In [8]:
combined_inv.head()

,BibNum,Title,Author,PublicationYear,Publisher,Subjects,ReportDate,ItemCount
0,4,Erotic art of the East; the sexual theme in or...,"Rawson, Philip S.",[1968],"Putnam,","Erotic art East Asia, Art Asian",09/01/2017,1
1,4,Erotic art of the East; the sexual theme in or...,"Rawson, Philip S.",[1968],"Putnam,","Erotic art East Asia, Art Asian",10/01/2017,1
2,7,"Birdless summer; China: autobiography, history.","Han, Suyin, 1917-2012",[1968],"Putnam,","Han Suyin 1917 2012, Authors Chinese 20th cent...",09/01/2017,1
3,7,"Birdless summer; China: autobiography, history.","Han, Suyin, 1917-2012",[1968],"Putnam,","Han Suyin 1917 2012, Authors Chinese 20th cent...",10/01/2017,1
4,12,Combat aircraft of the world; from 1909 to the...,"Taylor, John W. R. (John William Ransom), 1922...",[1969],Putnam,Airplanes Military,09/01/2017,1


In [9]:
merged = pd.merge(left=checkout_data, right = combined_inv, how='left', left_on='BibNumber', right_on='BibNum')
merged.head()

,BibNumber,ItemType,Collection,CheckoutDateTime,BibNum,Title,Author,PublicationYear,Publisher,Subjects,ReportDate,ItemCount
0,2543647,accd,nacd,01/02/2017 08:13:00 AM,2543647.0,Songs from a room [sound recording] / Leonard ...,"Cohen, Leonard, 1934-2016",p2007.,"Columbia/Legacy,",Popular music 1961 1970,09/01/2017,12.0
1,2543647,accd,nacd,01/02/2017 08:13:00 AM,2543647.0,Songs from a room [sound recording] / Leonard ...,"Cohen, Leonard, 1934-2016",p2007.,"Columbia/Legacy,",Popular music 1961 1970,10/01/2017,11.0
2,3172300,acbk,namys,01/02/2017 08:13:00 AM,3172300.0,I shot the Buddha / Colin Cotterill.,"Cotterill, Colin",[2016].,"Soho Crime,",Paiboun Siri Doctor Fictitious character Ficti...,09/01/2017,17.0
3,3172300,acbk,namys,01/02/2017 08:13:00 AM,3172300.0,I shot the Buddha / Colin Cotterill.,"Cotterill, Colin",[2016].,"Soho Crime,",Paiboun Siri Doctor Fictitious character Ficti...,10/01/2017,17.0
4,2393405,acbk,camys,01/02/2017 08:24:00 AM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
merged.drop('BibNum', axis=1, inplace=True)

In [11]:
merged.head()

,BibNumber,ItemType,Collection,CheckoutDateTime,Title,Author,PublicationYear,Publisher,Subjects,ReportDate,ItemCount
0,2543647,accd,nacd,01/02/2017 08:13:00 AM,Songs from a room [sound recording] / Leonard ...,"Cohen, Leonard, 1934-2016",p2007.,"Columbia/Legacy,",Popular music 1961 1970,09/01/2017,12.0
1,2543647,accd,nacd,01/02/2017 08:13:00 AM,Songs from a room [sound recording] / Leonard ...,"Cohen, Leonard, 1934-2016",p2007.,"Columbia/Legacy,",Popular music 1961 1970,10/01/2017,11.0
2,3172300,acbk,namys,01/02/2017 08:13:00 AM,I shot the Buddha / Colin Cotterill.,"Cotterill, Colin",[2016].,"Soho Crime,",Paiboun Siri Doctor Fictitious character Ficti...,09/01/2017,17.0
3,3172300,acbk,namys,01/02/2017 08:13:00 AM,I shot the Buddha / Colin Cotterill.,"Cotterill, Colin",[2016].,"Soho Crime,",Paiboun Siri Doctor Fictitious character Ficti...,10/01/2017,17.0
4,2393405,acbk,camys,01/02/2017 08:24:00 AM,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [12]:
merged.groupby('ReportDate').size()

ReportDate
09/01/2017    3371206
10/01/2017    3385551
dtype: int64

In [13]:
merged.tail()

,BibNumber,ItemType,Collection,CheckoutDateTime,Title,Author,PublicationYear,Publisher,Subjects,ReportDate,ItemCount
8400289,3208256,jcbk,cccomic,10/18/2017 07:58:00 PM,Princess Princess ever after / by Katie O'Neil...,"O'Neill, Katie (Cartoonist)",2016.,"Oni Press, Inc.,","Princesses Comic books strips etc, Cartoons an...",10/01/2017,12.0
8400290,2831364,accd,cacd,10/18/2017 07:58:00 PM,Spark seeker [sound recording] / Matisyahu.,Matisyahu,p2012.,"Fallen Sparks Records,","Reggae music, Popular music 2011 2020",09/01/2017,5.0
8400291,2831364,accd,cacd,10/18/2017 07:58:00 PM,Spark seeker [sound recording] / Matisyahu.,Matisyahu,p2012.,"Fallen Sparks Records,","Reggae music, Popular music 2011 2020",10/01/2017,5.0
8400292,3282121,acdvd,nadvd,10/18/2017 07:59:00 PM,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8400293,2486352,acdvd,nalndvd,10/18/2017 07:59:00 PM,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
merged[pd.isnull(merged).any(axis=1)]

,BibNumber,ItemType,Collection,CheckoutDateTime,Title,Author,PublicationYear,Publisher,Subjects,ReportDate,ItemCount
4,2393405,acbk,camys,01/02/2017 08:24:00 AM,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,3199718,acdvd,nadvdnf,01/02/2017 08:33:00 AM,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,3172511,acdvd,nadvd,01/02/2017 08:51:00 AM,NaN,NaN,NaN,NaN,NaN,NaN,NaN
17,3216974,acdvd,nadvd,01/02/2017 08:51:00 AM,NaN,NaN,NaN,NaN,NaN,NaN,NaN
20,3221781,acbk,nanf,01/02/2017 08:51:00 AM,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...
8400265,3245131,acbk,nycomic,10/18/2017 07:58:00 PM,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8400266,3263345,acdvd,cadvd,10/18/2017 07:58:00 PM,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8400287,2661830,accd,nacd,10/18/2017 07:58:00 PM,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8400292,3282121,acdvd,nadvd,10/18/2017 07:59:00 PM,NaN,NaN,NaN,NaN,NaN,NaN,NaN
